In [25]:
from neo4j import GraphDatabase
from neo4j.exceptions import Neo4jError

In [2]:
from dotenv import load_dotenv
import os

In [3]:
#Cargar variabvles de entorno del archivo .env
load_dotenv()

True

In [10]:
NEO4J_USER = os.getenv('N_USER')
NEO4J_PASSWORD = os.getenv('N_PASSWORD')

In [11]:
# Conexión a la base de datos Neo4j
uri = "neo4j+s://295c5d0b.databases.neo4j.io"
driver = GraphDatabase.driver(uri, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [30]:
# Consulta para extraer los nodos
def get_nodes(tx):
    query = "MATCH (n:articulos) RETURN n.articulo_id AS id, n.contenido AS contenido"
    result = tx.run(query)
    return [(record['id'], record['contenido']) for record in result]

In [31]:
with driver.session() as session:
    nodes = session.read_transaction(get_nodes)

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_9028\2132304414.py:2: DeprecationWarning: read_transaction has been renamed to execute_read
  nodes = session.read_transaction(get_nodes)


In [32]:
print(nodes[:10])

[(1, 'La presente ley regula la forma especial de dominio, denominado propiedad horizontal, en la que concurren derechos de propiedad exclusiva sobre bienes privados y derechos de copropiedad sobre el terreno y los demás bienes comunes, con el fin de garantizar la seguridad y la convivencia pacífica en los inmuebles sometidos a ella, así como la función social de la propiedad.NOTA: Texto subrayado declarado EXEQUIBLE por la Corte Constitucional mediante Sentencia C-318 de 2002, "bajo el entendido de que los moradores no propietarios de inmuebles sometidos al régimen de propiedad horizontal podrán ejercer ante las autoridades internas del mismo el derecho de petición, así como el de ser oídos en las decisiones que puedan afectarlos, conforme a lo expuesto en la parte motiva de esta sentencia.\'\'Igualmente, se declara la exequibilidad de las normas acusadas bajo el entendido de que, cuando se trate de la imposición de sanciones por parte de las autoridades internas del inmueble sometido

In [13]:
from sentence_transformers import SentenceTransformer

d:\0_Respaldo\000_KaggleX\Projects\Rag-Ley675\675_LAW\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [14]:
# Modelo optimizado para trabajar en espanol y para hacer preguntas.
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

d:\0_Respaldo\000_KaggleX\Projects\Rag-Ley675\675_LAW\.venv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USUARIO\.cache\huggingface\hub\models--sentence-transformers--paraphrase-xlm-r-multilingual-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

d:\0_Respaldo\000_KaggleX\Projects\Rag-Ley675\675_LAW\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [33]:
# Generar embeddings a partir del contenido de los nodos
ids, contenidos = zip(*nodes)
embeddings = model.encode(contenidos)

In [34]:
print(ids[:20])

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20)


In [35]:
# Guardar los embedings en la base de datos:
def store_embeddings(tx, node_id, embedding):
    query = """
    MATCH (n:articulos {id: $id})
    SET n.embedding = $embedding
    RETURN count(n) as updated
    """
    result = tx.run(query, id=node_id, embedding=embedding.tolist())
    return result.single()['updated']

def store_embeddings_batch(driver, ids, embeddings, batch_size=100):
    with driver.session() as session:
        for i in range(0, len(ids), batch_size):
            batch_ids = ids[i:i+batch_size]
            batch_embeddings = embeddings[i:i+batch_size]
            try:
                updated = session.write_transaction(
                    lambda tx: sum(store_embeddings(tx, node_id, emb) 
                                   for node_id, emb in zip(batch_ids, batch_embeddings))
                )
                print(f"Updated {updated} nodes in batch {i//batch_size + 1}")
            except Neo4jError as e:
                print(f"Error en el lote {i//batch_size + 1}: {str(e)}")

In [43]:
def store_embeddings(tx, node_id, embedding):
    # Primero, asegúrate de que el nodo exista
    query = """
    MATCH (n:articulos {articulo_id: $id})
    SET n.embedding = $embedding
    RETURN n.articulo_id AS id, n.embedding AS embedding
    """
    result = tx.run(query, id=node_id, embedding=embedding.tolist())
    return result.single()

In [44]:
with driver.session() as session:
    for node_id, embedding in zip(ids, embeddings):
        result = session.execute_write(store_embeddings, node_id, embedding)
        print(f"Stored Node ID: {result['id']}, Embedding: {result['embedding']}")

Stored Node ID: 1, Embedding: [0.172181636095047, 0.30855506658554077, 0.35117632150650024, 0.17206048965454102, -0.37834471464157104, 0.05786881968379021, -0.0985819548368454, 0.03666851669549942, 0.24202075600624084, 0.05629724636673927, -0.09143751859664917, 0.12900447845458984, 0.20157702267169952, 0.07168389856815338, -0.05582144111394882, -0.3946113884449005, 0.17840391397476196, -0.13225777447223663, 0.288420170545578, -0.35452353954315186, -0.00038114655762910843, -0.3150789737701416, 0.08727855980396271, 0.12200884521007538, 0.20721754431724548, -0.09830710291862488, 0.25589728355407715, 0.11675342172384262, -0.4147467017173767, 0.2546742856502533, -0.09667619317770004, -0.2607894837856293, -0.029364699497818947, 0.18532133102416992, 0.12317045778036118, 0.01611385866999626, 0.18158340454101562, -0.03521181270480156, 0.21953874826431274, -0.13552862405776978, 0.39788591861724854, 0.12087443470954895, -0.15979695320129395, 0.27681639790534973, 0.020515557378530502, 0.0193298235

# Similaridades

In [45]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity